In [67]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
import json

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [68]:
races = pd.read_csv(path+'data/races.csv')

weather = races.iloc[:,[0,1,2,3,4,5,6]]
weather.shape

(160, 7)

In [83]:
s2021 = weather.query('season == 2021 & round == 12')

race = s2021.iloc[0]

In [84]:
base = 'http://api.openweathermap.org/data/2.5/weather?'
api_key = open(path+'api_key.txt', 'r').read()
city = 'Cardiff'

race = weather.iloc[2]

lat = race.lat
lon = race.long

url = f"https://api.openweathermap.org/data/2.5/weather?appid={api_key}&lat={lat}&lon={lon}&units=metric"
response = requests.get(url)
data = json.loads(response.text)

print(data)

{'coord': {'lon': 50.5106, 'lat': 26.0325}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 34.94, 'feels_like': 37.41, 'temp_min': 34.94, 'temp_max': 34.94, 'pressure': 1010, 'humidity': 41, 'sea_level': 1010, 'grnd_level': 1008}, 'visibility': 10000, 'wind': {'speed': 5.95, 'deg': 150, 'gust': 6.34}, 'clouds': {'all': 54}, 'dt': 1667390406, 'sys': {'type': 1, 'id': 7397, 'country': 'BH', 'sunrise': 1667357222, 'sunset': 1667397354}, 'timezone': 10800, 'id': 290291, 'name': 'Bahrain', 'cod': 200}


In [86]:
print(data['weather'])
print(data['main'])
print(data['visibility'])
print(data['wind'])
# print(data['rain'])
print(data['clouds'])

[{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]
{'temp': 34.94, 'feels_like': 37.41, 'temp_min': 34.94, 'temp_max': 34.94, 'pressure': 1010, 'humidity': 41, 'sea_level': 1010, 'grnd_level': 1008}
10000
{'speed': 5.95, 'deg': 150, 'gust': 6.34}
{'all': 54}
